This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X'
r = requests.get(url)
json_data = r.json()


In [12]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(type(json_data))
print(json_data.keys())
print(json_data['dataset'].keys())

<class 'dict'>
dict_keys(['dataset'])
dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1. and 2. Making dictionary with usable 2017 AFX data#

AFX = json_data['dataset']
AFX_data = AFX['data']
columns = AFX['column_names']
#columns: 'Change', 'Last Price of the Day','Daily Traded Units','Daily Turnover' were empty for AFX in 2017 so they were removed#
clean_columns = ['Date','Open','High','Low','Close', 'Traded Volume', 'Turnover']
data_2017 = []
dict_2017 = {}

for i in AFX_data:
    if '2017' in i[0]:
        data_2017.append(i)

for n in clean_columns:
    dict_2017[n] = []

for d in data_2017:
    dict_2017['Date'].append(d[0])
    dict_2017['Open'].append(d[1])
    dict_2017['High'].append(d[2])
    dict_2017['Low'].append(d[3])
    dict_2017['Close'].append(d[4])
    dict_2017['Traded Volume'].append(d[6])
    dict_2017['Turnover'].append(d[7])
 

In [6]:
#3. Calculating highest and lowest opening price for AFX in 2017#
Open_lst = dict_2017['Open']
#there were a couple None type objects for open price in 2017 so these were removed#
cln_open = list(filter(None, Open_lst))

min_open = min(cln_open)
max_open = max(cln_open)

print(f'During 2017, the lowest opening price was ${min_open} and highest opening price was ${max_open}.')


During 2017, the lowest opening price was $34.0 and highest opening price was $53.11.


In [29]:
#4. Calculating largest change in price for one day #
high_lst = dict_2017['High']
low_lst = dict_2017['Low']
diff_lst = []

for n in range(255):
    diff_lst.append(high_lst[n]-low_lst[n])
    
MaxDiff = max(diff_lst)
max_idx = diff_lst.index(MaxDiff)
date = dict_2017['Date'][max_idx]

print(f'The largest change in price for any given day was ${MaxDiff:.2f}, which occured on {date}.')

The largest change in price for any given day was $2.81, which occured on 2017-05-11.


In [43]:
#5. Calculating largest change between 2 subsequent closing prices#
close_lst = dict_2017['Close']
close_diff = []

for c in range(1,255):
    close_diff.append(close_lst[c]-close_lst[c-1])
max_pos = max(close_diff)  
max_neg = min(close_diff)

if abs(max_pos) > abs(max_neg):
    print(f'The largest closing price change between consecutive days was an increase of ${max_pos:.2f}.')
if abs(max_pos) < abs(max_neg):
    print(f'The largest closing price change between consecutive days was an decrease of ${max_neg:.2f}.')


The largest closing price change between consecutive days was an increase of $2.56.


In [47]:
#6. Calculating average daily trading volume#
vol_lst = dict_2017['Traded Volume']
avg = sum(vol_lst) / len(vol_lst)

print(f'During 2017, the average daily trading volume was {avg:.0f}')

During 2017, the average daily trading volume was 89124


In [71]:
#7. Calculating median daily trading volume#

#simple function created to find medians from a list 
def median(lst):
    sorted_lst = sorted(lst)
    med_idx = int(len(lst) / 2) 
    if len(lst) % 2 == 1:
        return lst[med_idx]
    else:
        return (lst[med_idx] + lst[med_idx-1]) / 2
        
print(f'The median daily trading volume for 2017 was {median(vol_lst)}.')

The median daily trading volume for 2017 was 134965.0.
